In [2]:
import pyathena as pa

[stellar-intel.princeton.edu:3878898] mca_base_component_repository_open: unable to open mca_op_avx: /usr/local/openmpi/4.1.0/gcc/lib64/openmpi/mca_op_avx.so: undefined symbol: ompi_op_base_module_t_class (ignored)


In [3]:
sim=pa.LoadSimTIGRESSNCR('/scratch/gpfs/changgoo/TIGRESS-NCR/R8_8pc_NCR.full.b1.v3.iCR4.Zg0.01.Zd0.01.xy4096.eps0.0')

In [4]:
ds = sim.load_vtk(sim.nums[-1])

In [6]:
ds.field_list

['density',
 'velocity',
 'pressure',
 'cell_centered_B',
 'gravitational_potential',
 'temperature',
 'heat_rate',
 'cool_rate',
 'net_cool_rate',
 'CR_ionization_rate',
 'rad_energy_density_PH',
 'rad_energy_density_LW',
 'rad_energy_density_PE',
 'rad_energy_density_PE_unatt',
 'rad_energy_density_LW_diss',
 'specific_scalar[0]',
 'specific_scalar[1]',
 'xHI',
 'xH2',
 'xe']

In [24]:
data = ds.get_field(["nH","T","xHI","xH2","xHII","xe","specific_scalar[0]","ne"])

In [25]:
data = data.rename({"specific_scalar[0]":"Z"})

In [26]:
data

<xarray.Dataset>
Dimensions:  (z: 768, y: 128, x: 128)
Coordinates:
  * x        (x) float64 -508.0 -500.0 -492.0 -484.0 ... 484.0 492.0 500.0 508.0
  * y        (y) float64 -508.0 -500.0 -492.0 -484.0 ... 484.0 492.0 500.0 508.0
  * z        (z) float64 -3.068e+03 -3.06e+03 -3.052e+03 ... 3.06e+03 3.068e+03
Data variables:
    xHI      (z, y, x) float32 0.001451 0.002443 ... 7.785e-06 9.927e-06
    xH2      (z, y, x) float32 4.123e-14 1.981e-13 5.259e-13 ... 1e-20 1e-20
    xe       (z, y, x) float32 0.9986 0.9976 0.9968 0.9965 ... 1.184 1.18 1.17
    Z        (z, y, x) float32 0.00566 0.00561 0.005562 ... 0.008214 0.00873
    ne       (z, y, x) float32 3.181e-08 3.682e-08 ... 5.722e-06 3.741e-06
    xHII     (z, y, x) float32 0.9985 0.9976 0.9968 0.9965 ... 1.0 1.0 1.0 1.0
    T        (z, y, x) float32 393.2 349.2 576.4 ... 1.008e+05 9.086e+04
    nH       (z, y, x) float32 3.186e-08 3.691e-08 ... 4.849e-06 3.198e-06
Attributes:
    all_grid_equal:  True
    ngrid:           384
    le:              [ -512.  -512. -3072.]
    re:              [ 512.  512. 3072.]
    dx:              [8. 8. 8.]
    Lx:              [1024. 1024. 6144.]
    center:          [0. 0. 0.]
    Nx:              [128 128 768]
    ndim:            3
    time:            1000.0
    dfi:             {'rho': {'field_dep': ['density'], 'func': <function set...

In [27]:
from pyathena.tigress_ncr.phase import assign_phase

In [28]:
phase = assign_phase(sim,data,kind="five2")

In [29]:
phase.max()

<xarray.DataArray 'phase' ()>
array(4)

In [40]:
Z_proj_EM=dict()
Z_proj_rho=dict()
for i, ph in enumerate(phase.attrs["phlist"]):
    EM = (phase == i)*data["ne"]**2
    rho = (phase == i)*data["nH"]
    # EM weighted mean Z
    Z_proj_EM[ph] = (EM*data["Z"]).sum(dim="z")/0.02/EM.sum(dim="z")
    # density weighted mean Z
    Z_proj_rho[ph] = (rho*data["Z"]).sum(dim="z")/0.02/rho.sum(dim="z")

EM = data["ne"]**2
rho = data["nH"]
# EM weighted mean Z
Z_proj_EM["all"] = (EM*data["Z"]).sum(dim="z")/0.02/EM.sum(dim="z")
# density weighted mean Z
Z_proj_rho["all"] = (rho*data["Z"]).sum(dim="z")/0.02/rho.sum(dim="z")

In [57]:
import os
dname = os.path.join(sim.savdir,"Zprj")
os.makedirs(dname,exist_ok=True)


In [58]:
fname = os.path.join(dname,f"{sim.basename}.{sim.nums[-1]:04d}.Zprj.nc")
print(fname)

/scratch/gpfs/changgoo/TIGRESS-NCR/R8_8pc_NCR.full.b1.v3.iCR4.Zg0.01.Zd0.01.xy4096.eps0.0/Zprj/R8_8pc_NCR.full.b1.v3.iCR4.Zg0.01.Zd0.01.xy4096.eps0.0.1000.Zprj.nc


In [41]:
import xarray as xr

dset = xr.Dataset()
for Z_proj in [Z_proj_EM,Z_proj_rho]:
    dset["EM"] = xr.Dataset(Z_proj).to_array("phase")
    dset["rho"] = xr.Dataset(Z_proj).to_array("phase")
dset.to_netcdf(fname)